In [1]:
%pip install neo4j


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from neo4j import GraphDatabase

# Classe para lidar com a conexão e transações do Neo4j
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def execute_query(self, query, **kwargs):
        with self._driver.session() as session:
            result = session.execute_write(self._execute, query, **kwargs)
            return result

    @staticmethod
    def _execute(tx, query, **kwargs):
        result = tx.run(query, **kwargs)
        return result.data()

# Configurações de conexão com o banco de dados Neo4j
uri = "neo4j+s://f08621b4.databases.neo4j.io"  # Substitua com a URL correta do seu banco de dados
user = "neo4j"           # Substitua com seu nome de usuário
password = "UtRsFSYanL1-Mvj1uZ4War-uiW-oicu-y8LizZoLbpA"         # Substitua com sua senha

# Exemplo de uso

connection = Neo4jConnection(uri, user, password)

# Consulta de exemplo
query = "MATCH (n) RETURN n LIMIT 5"

try:
    result = connection.execute_query(query)
    for record in result:
        print(record)
except Exception as e:
    print(f"Erro: {str(e)}")
finally:
    connection.close()


In [35]:
class Player:
    def __init__(self, nome, id):
        self.nome = nome
        self._id = id
        connection = Neo4jConnection(uri, user, password)
        try:
            result = connection.execute_query(f"CREATE (p:Player {{id: {self._id}, nome: '{self.nome}'}})")
        except Exception as e:
            print(str(e))
        finally:
            connection.close()

    def Update(self, n_nome, n_id):
        try:
           
            result = connection.execute_query(f"MATCH (p:Player {{id: {self._id}, nome: '{self.nome}'}}) set p.nome = '{n_nome}' , p.id = {n_id}")
            self.nome = n_nome
            self._id = n_id
            
        except Exception as e:
            print(str(e))
        finally:
            connection.close()

    def Delete(self):
        try:
            result = connection.execute_query(f"MATCH(n:Player{{id : {self._id}}}) DELETE n")
        except Exception as e:
            print(str(e))
        finally:
            connection.close()
    def get_history(self):
        try:
            result = connection.execute_query(f"MATCH (p:Player {{id:{self._id}}})-[r:PARTICIPATED_IN]-> (m:Match) RETURN m, r.score")
            print(result)
        except Exception as e:
            print(str(e))
        finally:
            connection.close()


        
    
class Match:
    def __init__(self, id):
        self._id = id
        connection = Neo4jConnection(uri, user, password)
        try:
            result = connection.execute_query(f"CREATE (m:Match {{id: {self._id}}})")
        except Exception as e:
            print(str(e))
        finally:
            connection.close()
    
    def add_player(self, player:Player, score):
        try:
            result = connection.execute_query(f"MATCH(p:Player{{id : {player._id}}}) MATCH(m:Match{{id:{self._id}}}) CREATE (p)-[:PARTICIPATED_IN {{score:{score}}}]->(m)")
        except Exception as e:
            print(str(e))
        finally:
            connection.close()

    def player_list(self):
        try:
            result = connection.execute_query(f"MATCH (player:Player)-[r:PARTICIPATED_IN]-> (match:Match{{id:{self._id}}}) RETURN p")
            print(result)
        except Exception as e:
            print(str(e))
        finally:
            connection.close()

        

    

partida1 = Match(1)
pl1 = Player("Jogador a ", 1)
pl2 = Player("Jogador b ", 2)
pl3 = Player("Jogador c ", 3)

partida1.add_player(pl1,10)
partida1.add_player(pl3,6)

partida2 = Match(2)
partida2.add_player(pl2,5)
partida2.add_player(pl1,8)

pl1.get_history()
partida1.player_list()





C:\Users\mateu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  if sys.path[0] == "":


[{'m': {'id': 2}, 'r.score': 8}, {'m': {'id': 1}, 'r.score': 10}]
[{'p': {'nome': 'Jogador c ', 'id': 3}}, {'p': {'nome': 'Jogador a ', 'id': 1}}]
